<a href="https://colab.research.google.com/github/mifm/examples/blob/master/XRSite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install py_wake

In [24]:
import xarray as xr
from py_wake.site import XRSite
from py_wake.examples.data.hornsrev1 import Hornsrev1Site,V80, wt_x, wt_y, wt16_x, wt16_y
from py_wake import NOJ

windTurbines = V80()

# import a site example just to get some probability data
site = Hornsrev1Site()

# create your custom data set
p_il = site.local_wind(0,0).P
wd = site.local_wind(0,0).wd
ws = site.local_wind(0,0).ws
ds = xr.Dataset(
    data_vars={'P': (['wd', 'ws'], p_il),
               'TI': site.local_wind(0,0).TI},
    coords={'wd': wd,
            'ws': ws})

# create a site from a custom data set
mysite = XRSite(ds)

You can see the probability for the site you have defined does not exactly sum to 1. You can use the normalize_probabilities keyword to do this.

In [37]:
noj = NOJ(mysite,windTurbines)
print('P = ', float(mysite.ds.P.sum()))
print('AEP_norm = ',noj.aep(site.initial_position[:,0], site.initial_position[:,1], normalize_probabilities=True))
print('AEP = ',noj.aep(site.initial_position[:,0], site.initial_position[:,1], normalize_probabilities=False))

P =  0.9747995988520103
AEP_norm =  721.4431676287484
AEP =  702.4351577190602


Sensitivity to bin size. If you reduce the bin size of wind directions (only taking every 2nd element of the wind direction tensor and probability tensor in this example. Use proper downsampling normally.) you can recalculate the aep.

In [32]:
wd2 = wd[::2]
p_il2 = p_il[::2,:] * 2

In [34]:
mysite2 = XRSite(xr.Dataset(
    data_vars={'P': (['wd', 'ws'], p_il2),
               'TI': site.local_wind(0,0).TI},
    coords={'wd': wd2,
            'ws': ws}))

In [38]:
noj2 = NOJ(mysite2,windTurbines)
print('P = ', float(mysite2.ds.P.sum()))
print('AEP_norm = ',noj2.aep(site.initial_position[:,0], site.initial_position[:,1], normalize_probabilities=True))
print('AEP = ',noj2.aep(site.initial_position[:,0], site.initial_position[:,1], normalize_probabilities=False))

P =  0.9759464011464817
AEP_norm =  721.4437392283411
AEP =  702.4357142586023
